In [ ]:
from bertopic import BERTopic
import json
import pandas as pd
from  services.influencers_service import  InfluencersService
from shared.mongo import MongoConnection
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
mongo_connection=MongoConnection()
influencers_service = InfluencersService(mongo_connection)
bios=[]
posts=influencers_service.get_posts()
influencers=influencers_service.get_influencers()
custom_stopwords={'null','like','com','www','gmail','hotmail'}
for influencer in influencers:
    info_string=""
    if influencer['Bio']!= 'NULL':
        info_string+=influencer['Bio']
    if influencer['Category']!= 'NULL':
        info_string+=influencer['Category']
    if info_string!= "":
        for stopword in custom_stopwords:
            info_string=info_string.replace(stopword,"")
        bios.append(info_string)

In [ ]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")


In [ ]:
topics, probs = topic_model.fit_transform(bios)
print(topics)
print(probs)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# representation_model = KeyBERTInspired()
# topic_model = BERTopic(representation_model=representation_model)
# df = pd.DataFrame({'topic': topics, 'document': bios})
# topic_model.reduce_topics(bios, nr_topics=100)
# topic_model.merge_topics(bios,[0,5])
# vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
# topic_model.update_topics(bios, vectorizer_model=vectorizer_model)

In [ ]:
# topic_model.save("/models/bert_9", serialization="safetensors")
topic_model=BERTopic.load("/models/bert_7",embedding_model="all-MiniLM-L6-v2")

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
# topic_model.get_topic_info(30)
topic_model.get_topic(topic=5)
# topic_model.get_representative_docs(30)

In [ ]:
new_topics, new_prob=topic_model.transform(documents="vogue")
print("topic_id:",new_topics,"probabilty of correct prediction:",new_prob)
#note : [-1] [.0] means the model couldn't fit the data in any cluster

In [ ]:
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

topic_model = BERTopic(verbose=True, n_gram_range=(1, 3))
topics, _ = topic_model.fit_transform(bios)

# Preprocess Documents
documents = pd.DataFrame({"Document": bios,
                          "ID": range(len(bios)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)